STEP 1: IMPORT LIBRARIES

In [5]:
%pip install groq

  Obtaining dependency information for groq from https://files.pythonhosted.org/packages/d6/7d/877dbef7d72efacc657777b2e7897baa7cc7fcd0905f1b4a6423269e12a1/groq-0.31.1-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/134.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/134.9 kB ? eta -:--:--
   -------- ------------------------------ 30.7/134.9 kB 660.6 kB/s eta 0:00:01
   ----------- --------------------------- 41.0/134.9 kB 281.8 kB/s eta 0:00:01
   ----------- --------------------------- 41.0/134.9 kB 281.8 kB/s eta 0:00:01
   ----------- --------------------------- 41.0/134.9 kB 281.8 kB/s eta 0:00:01
   ----------- --------------------------- 41.0/134.9 kB 281.8 kB/s eta 0:00:01
   -------------------- ------------------ 71.7/134.9 kB

In [16]:
import pandas as pd
from groq import Groq

STEP 2: CONFIG

In [ ]:
API_KEY = "YOUR_GROQ_KEY"
DATASET_PATH = "customer_support_tickets.csv"  
OUTPUT_PATH = "ticket_results.csv"

In [18]:
CATEGORIES = ["Billing", "Technical Issue", "Account Management", "Password Reset", "Other"]

In [19]:
FEW_SHOT_EXAMPLES = [
    ("I was charged twice on my credit card.", "Billing"),
    ("My internet keeps disconnecting.", "Technical Issue"),
    ("I want to change the email linked to my account.", "Account Management"),
    ("I forgot my password and need to reset it.", "Password Reset"),
]

In [24]:
client = Groq(api_key=API_KEY)
MODEL_ID = "llama-3.3-70b-versatile"

STEP 3: CLASSIFY ZERO SHOT

In [25]:
def classify_zero_shot(ticket):
    prompt = f"""
You are a support ticket classifier.
Categories: {", ".join(CATEGORIES)}

Ticket: "{ticket}"

Return the top 3 most likely categories with confidence scores (0 to 1).
Format: Category - Score
    """
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

STEP 4 : CLASSIFY FEW SHOT

In [26]:
def classify_few_shot(ticket):
    examples_str = "\n".join([f"Ticket: {ex[0]} → Category: {ex[1]}" for ex in FEW_SHOT_EXAMPLES])
    prompt = f"""
You are a support ticket classifier.
Categories: {", ".join(CATEGORIES)}

Here are some examples:
{examples_str}

Now classify this new ticket and return the top 3 most likely categories with confidence scores (0 to 1).

Ticket: "{ticket}"
Format: Category - Score
    """
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


STEP 5: MAIN WORKFLOW

In [27]:
def main():
    df = pd.read_csv(DATASET_PATH)
    if "Ticket Description" not in df.columns:
        raise ValueError("Dataset must have a 'Ticket Description' column.")

    results = []
    for ticket in df["Ticket Description"].head(20):
        zs = classify_zero_shot(ticket)
        fs = classify_few_shot(ticket)
        results.append({
            "Ticket": ticket,
            "Zero-Shot Prediction": zs,
            "Few-Shot Prediction": fs
        })
        print(f"\nTICKET: {ticket}\nZERO-SHOT: {zs}\nFEW-SHOT: {fs}")

    pd.DataFrame(results).to_csv(OUTPUT_PATH, index=False)
    print(f"\n✅ Results saved to {OUTPUT_PATH}")

if __name__ == "__main__":
    main()


TICKET: I'm having an issue with the {product_purchased}. Please assist.

Your billing zip code is: 71701.

We appreciate that you have requested a website address.

Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.
ZERO-SHOT: Technical Issue - 0.8
Account Management - 0.1
Other - 0.1 

 Reasoning: The ticket mentions a specific issue with the product purchased and troubleshooting steps from the user manual, indicating a technical problem. Although there's a mention of a billing zip code, it seems to be incidental information and not the primary concern of the ticket. The mention of a website address and email address also suggests some account or configuration aspect, but it's not the main focus.
FEW-SHOT: Based on the text, here are the top 3 most likely categories with confidence scores:

Technical Issue - 0.8
Account Management - 0.1
Billing - 0.1

The ticket mentions a specific issue with a product, trou